In [29]:
from lusidtools.jupyter_tools import toggle_code

"""Exchange Traded Option - Valuation Workflow

Attributes
----------
exchange traded options
options on bond futures
transaction types
recipes
valuations
"""

toggle_code("Toggle Docstring")

# Booking and valuing an Exchange Traded Option

In this notebook, we demonstrate how an exchange traded option can be booked in LUSID, and extend it to use cases such as running a standard valuation and calculating PnL.

**Table of Contents:**
-  [1. Create Portfolio](#1.-Create-Portfolio)
-  [2. Create Instrument](#2.-Create-Instrument)
-  [3. Upsert Transactions](#3.-Upsert-Transactions)
-  [4. Upsert Quotes](#4.-Upsert-Quotes)
-  [5. Create Configuration Recipe](#5.-Create-Configuration-Recipe)
-  [6. Valuation](#6.-Valuation)

In [30]:
# Import generic non-LUSID packages
import os
import pandas as pd
from datetime import datetime, timedelta
from dateutil.parser import parse
import json
import pytz
from IPython.core.display import HTML

# Import key modules from the LUSID package
import lusid
import lusid.models as lm
import lusid.api as la
from lusid.utilities import ApiClientFactory

# Import key functions from Lusid-Python-Tools and other packages
from lusidjam import RefreshingToken

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

# For running the notebook locally
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Authenticate our user and create our API client
api_factory = ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path)

# Set pandas dataframe display formatting
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

print ('LUSID Environment Initialised')
print ('LUSID API Version :', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

# Set APIs
portfolio_api = api_factory.build(lusid.api.PortfoliosApi)
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
aggregration_api = api_factory.build(lusid.api.AggregationApi)

LUSID Environment Initialised
LUSID API Version : 0.6.8179.0


In [31]:
# Define scopes
scope = "ibor"
market_data_scope = "ibor"

# 1. Create Portfolio

Create the portfolio that will contain the option position.

In [32]:
portfolio_code = "ExchangeTradedOption"

try:
    transaction_portfolios_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency="USD",
            created="2010-01-01",
            sub_holding_keys=[],
        ),
    )

except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

Could not create a portfolio with id ExchangeTradedOption because it already exists in scope ibor.


# 2. Create Instrument

Using the below method we can reference the SDK models to populate the required parameters for creating an exchange traded option. We then populate the instrument variables, and make a call to persist the instrument in LUSID.

In [33]:
# Define function that creates futures
def create_exchange_option(
        description,
        start_date,
        exercise_date,
        dom_ccy,
        strike,
        contract_size,
        country_id,
        delivery_type,
        exchange_code,
        exercise_type,
        option_code,
        option_type,
        underlying_code,
        ref_spot_price,
):
    contract_details = lm.ExchangeTradedOptionContractDetails(
        dom_ccy=dom_ccy,
        strike=strike,
        contract_size=contract_size,
        country=country_id,
        delivery_type=delivery_type,
        description=description,
        exchange_code=exchange_code,
        exercise_date=exercise_date,
        exercise_type=exercise_type,
        option_code=option_code,
        option_type=option_type,
        underlying=lm.ExoticInstrument(
            instrument_format=lm.InstrumentDefinitionFormat(
                "custom", "custom", "0.0.0"
            ),
            content="{}",
            instrument_type="ExoticInstrument",
        ),
        underlying_code=underlying_code,
    )
    
    return lm.ExchangeTradedOption(
        start_date=start_date,
        contract_details=contract_details,
        contracts=1,
        ref_spot_price=ref_spot_price,

        instrument_type="ExchangeTradedOption",
    )


# Define a function to upsert instrument
def upsert_instrument_to_lusid(instrument_definition, name, identifier, identifier_type):
        return instruments_api.upsert_instruments(
            request_body={
                identifier: lm.InstrumentDefinition(
                    name=name,
                    identifiers={
                        identifier_type: lm.InstrumentIdValue(value=identifier)
                    },
                    definition=instrument_definition,
                )
            }
        )


In this example, we will be looking to create a CME call option with an underlying of a 10Y US Treasury bond future. 

- Underlying: US 10Y Treasury Bond Future
- Strike: 130.25
- Option Type: Call
- Excercise Type: American
- Contract Size: USD 100,000

In [34]:
# Set the instrument variables
option_name = "US BOND FUTR OPTN Dec21C"
option_identifier = "OPT-USZ1C15721"
identifier_type = "ClientInternal"
description = "130.25 Call US 10Y Bond Future"
start_date = datetime(2021, 2, 22, tzinfo=pytz.utc)
exercise_date= datetime(2021, 10, 29, tzinfo=pytz.utc)
dom_ccy = "USD"
strike = 130.25
contract_size=100000
country_id = "US"
delivery_type = "Cash"
exchange_code = "CBOT"
exercise_type = "American"
option_code = "US"
option_type = "Call"
underlying_code = "TY"
contracts = 1
ref_spot_price = 130.875

In [35]:
# Create the Instrument
option_definition = create_exchange_option(
    description=description,
    start_date=start_date,
    exercise_date=exercise_date,
    dom_ccy=dom_ccy,
    strike=strike,
    contract_size=contract_size,
    country_id=country_id,
    delivery_type=delivery_type,
    exchange_code=exchange_code,
    exercise_type=exercise_type,
    option_code=option_code,
    option_type=option_type,
    underlying_code=underlying_code,
    ref_spot_price=ref_spot_price,
)

# Upsert the instrument
upsert_response = upsert_instrument_to_lusid(option_definition, option_name, option_identifier, identifier_type)
luid = upsert_response.values[option_identifier].lusid_instrument_id
print(luid)

LUID_0000BOXN


# 3. Upsert transactions

Having created the portfolio, we add a simple _Buy_ transaction against the option to create a position. In this case we book the total consideration or cost of the transaction as the cost amount, whereas the premium is going to be booked as a reference price for the transaction.

In [36]:
# Set trade variables
trade_date = datetime(2021, 9, 25, tzinfo=pytz.utc)
settle_days = 2
premium = 0.859375
units = 100
cost_amount = premium/100 * units * contract_size

# Book a buy transaction against the option
opt_txn = lm.TransactionRequest(
    transaction_id="TXN001",
    type="Buy",
    instrument_identifiers={"Instrument/default/ClientInternal": option_identifier},
    transaction_date=trade_date,
    settlement_date=trade_date + timedelta(days=settle_days),
    units=units,
    transaction_price=lm.TransactionPrice(price=premium,type="Price"),
    total_consideration=lm.CurrencyAndAmount(amount=cost_amount,currency="USD"),
    exchange_rate=1,
    transaction_currency="USD"
)

response = transaction_portfolios_api.upsert_transactions(scope=scope,
                                                    code=portfolio_code,
                                                    transaction_request=[opt_txn])

print(f"Transaction succesfully updated at time: {response.version.as_at_date}")

Transaction succesfully updated at time: 2021-10-27 14:39:36.330481+00:00


# 4. Upsert Quotes

In order to conduct a valuation, we will be loading prices for the option into the market data store. In this particular example, we are simply using quotes provided by the exchange which represent the daily value of the option contract. For exchange traded options, these are readily available for contracts that have non-zero open interest.

Notice the premiums should represent percentages, so we also add a `scale_factor` of a 100 to each quote.

In [37]:
# Read quotes and make datetimes timezone aware
quotes_df = pd.read_csv("data/opt_quotes.csv")
quotes_df["Date"] = quotes_df["Date"].apply(lambda x: parse(x).replace(tzinfo=pytz.utc))
quotes_df.head()

,Date,Price,Identifier
0,2021-09-25 00:00:00+00:00,0.86,OPT-USZ1C15721
1,2021-09-26 00:00:00+00:00,0.84,OPT-USZ1C15721
2,2021-09-27 00:00:00+00:00,0.91,OPT-USZ1C15721
3,2021-09-28 00:00:00+00:00,0.90,OPT-USZ1C15721
4,2021-09-29 00:00:00+00:00,0.90,OPT-USZ1C15721


In [38]:
# Set the quote scale factor to 100
scale_factor = 100

# Create quotes request
instrument_quotes = {
            index: lm.UpsertQuoteRequest(
            quote_id=lm.QuoteId(
                quote_series_id=lm.QuoteSeriesId(
                    provider="Lusid",
                    instrument_id=row["Identifier"],
                    instrument_id_type="ClientInternal",
                    quote_type="Price",
                    field="mid",
                ),
                effective_at=row["Date"],
            ),
            metric_value=lm.MetricValue(value=row["Price"], unit="USD"),
            scale_factor=scale_factor
        )
    for index, row in quotes_df.iterrows()
}

# Upsert quotes into LUSID
response = quotes_api.upsert_quotes(
    scope=market_data_scope, request_body=instrument_quotes
)

if response.failed == {}:
    print(f"Quotes successfully loaded into LUSID. {len(response.values)} quotes loaded.")
else:
    print(f"Some failures occurred during quotes upsertion, {len(response.failed)} did not get loaded into LUSID.")

Quotes successfully loaded into LUSID. 5 quotes loaded.


# 5. Create configuration recipe

Next, we will create a configuration recipe that will drive how the valuation is conducted. That is, where to derive quotes from in the market data store and what valuation method to use.

In case of an exchange traded option, the natural model choice is `SimpleStatic`, as prices are readily available from the exchange. Within the market context, we set the sources of market data to be used.

This will result in the following computation for the option's market value on a given day:

- `PV = P /SF * Q`
- `P = Price`
- `SF = Scale Factor`
- `Q = Quantity or Notional`

It is worth noting a quote's default scale factor is 1 when not specified, however for the listed option in question a value of `100` was provided.


In [39]:
# Set recipe code
recipe_code = "exchangeTradedOptValuation"

# Populate recipe parameters
configuration_recipe = lm.ConfigurationRecipe(
    scope=scope,
    code=recipe_code,
    market=lm.MarketContext(
        market_rules=[
            lm.MarketDataKeyRule(
                key="Equity.ClientInternal.*",
                supplier="Lusid",
                data_scope=market_data_scope,
                quote_type="Price",
                field="mid",
                quote_interval="5D.0D"
            )
        ],
    ),
    pricing=lm.PricingContext(
        model_rules=[
            lm.VendorModelRule(
                supplier="Lusid",
                model_name="SimpleStatic",
                instrument_type="ExchangeTradedOption",
                parameters="{}",
            )
        ]
    ),
)

response = configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=lm.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )


print(f"Configuration recipe loaded into LUSID at time {response.value}.")

Configuration recipe loaded into LUSID at time 2021-11-17 16:24:24.048433+00:00.


# 6. Valuation

For valuing the portfolio, we will need to create a valuation request that specifies the metrics that are to be returned, and reference our previously setup portfolio and recipe on a given date. The following method will return the results in form of a data frame.

In [40]:
def get_daily_valuation(date, portfolio_code):

    valuation_request = lm.ValuationRequest(
        recipe_id=lm.ResourceId(scope=scope, code=recipe_code),
        metrics=[
            lm.AggregateSpec("Instrument/default/Name", "Value"),
            lm.AggregateSpec("Instrument/default/ClientInternal", "Value"),
            lm.AggregateSpec("Instrument/Definition/ContractSize", "Value"),
            lm.AggregateSpec("Quotes/Price", "Value"),
            lm.AggregateSpec("Holding/default/Units", "Value"),
            lm.AggregateSpec("Valuation/PV/Amount", "Value"),
            lm.AggregateSpec("Valuation/Exposure", "Value"),
            lm.AggregateSpec("Valuation/PnL/Tm1", "Value"),
        ],
        group_by=["Instrument/default/Name"],
        portfolio_entity_ids=[
            lm.PortfolioEntityId(scope=scope, code=portfolio_code)
        ],
        valuation_schedule=lm.ValuationSchedule(effective_at=date),
    )

    val_data = aggregration_api.get_valuation(valuation_request=valuation_request).data

    vals_df = pd.DataFrame(val_data)

    vals_df.rename(
        columns={
            "Instrument/default/Name": "InstrumentName",
            "Instrument/default/ClientInternal": "ClientInternal",
            "Valuation/PV": "Value",
            "Valuation/PnL/Tm1": "PnL (1-day)",
        },
        inplace=True,
    )

    return vals_df

# 6.1 Daily Valuation and PnL

# Day 1

We begin by valuing the position as of the trade date, where based on the available quotes we can observe the following market value for our option. Notice, that given the daily quote is the same as the cost amount quoted in our transaction, we also have a matching cash outflow within our portfolio.

In [41]:
df = get_daily_valuation(trade_date, portfolio_code)
df.drop('Aggregation/Errors', axis=1, inplace=True)
df

,Valuation/PV/Amount,InstrumentName,ClientInternal,Instrument/Definition/ContractSize,Quotes/Price,Holding/default/Units,Valuation/Exposure,PnL (1-day)
0,"85,937.50",US BOND FUTR OPTN Dec21C,OPT-USZ1C15721,"100,000.00",0.86,100.00,"85,937.50",nan
1,"-85,937.50",USD,None,1.00,nan,"-85,937.50","-85,937.50",0.00


# Day 2

On the following date, we observe a new market value and also observe a daily PnL as a result of a price change against the previous day's market value. We also query the price on the quotes store for the security, where we observe a new price of 0.84 resulting in the loss.

In [42]:
df = get_daily_valuation(trade_date + timedelta(days=1), portfolio_code)
df

,Valuation/PV/Amount,InstrumentName,ClientInternal,Instrument/Definition/ContractSize,Quotes/Price,Holding/default/Units,Valuation/Exposure,PnL (1-day)
0,"84,375.00",US BOND FUTR OPTN Dec21C,OPT-USZ1C15721,"100,000.00",0.84,100.00,"84,375.00","-1,562.50"
1,"-85,937.50",USD,None,1.00,nan,"-85,937.50","-85,937.50",0.00


# Day 3

On day 3, we again run the same query and observe a daily gain, as well as the latest market value and price.

In [43]:
df = get_daily_valuation(trade_date + timedelta(days=2), portfolio_code)
df


,Valuation/PV/Amount,InstrumentName,ClientInternal,Instrument/Definition/ContractSize,Quotes/Price,Holding/default/Units,Valuation/Exposure,PnL (1-day)
0,"90,625.00",US BOND FUTR OPTN Dec21C,OPT-USZ1C15721,"100,000.00",0.91,100.00,"90,625.00","6,250.00"
1,"-85,937.50",USD,None,1.00,nan,"-85,937.50","-85,937.50",0.00
